In [3]:
import pandas as pd
import numpy as np

#gensim
#pip install --upgrade gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel

#nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

# Plotting tools
#pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from sklearn.utils import resample

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/janestout/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_csv('data/movies/tags.csv', low_memory=False)

# df.suicide.value_counts()

FileNotFoundError: File b'data/movies/tags.csv' does not exist

In [129]:
# df_majority = df[df.suicide==0]
# df_minority = df[df.suicide==1]
# #
# df_majority_downsampled = resample(df_majority, replace=False, n_samples=6633, random_state=123)
# df_downsampled = pd.concat([df_majority_downsampled, df_minority])
# df=df_downsampled.copy()

# df.suicide.value_counts()

In [130]:
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 200
# df['motive'].replace(to_replace=['Unknown', 'The specific motive for the attack is unknown.'],value=np.NaN, inplace=True)
data = df[['tag']]
data.dropna(inplace = True)
data_text = data[['motive']]
# data_text['index'] = data_text.index
documents = data_text
# print(len(documents))
print(documents[:70])

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       motive
3    The specific motive is unknown; however, sources noted that Conrado Oligo Agtarap may have been targeted because he was running for town councilor.                                                                                                                                                                                                                                                                                                  

/Users/janestout/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [131]:
my_stopwords = {'motive', 'attack', 'Unknown', 'unknown', 'however', 'sources', 'specific', 'stated', 'statement', 'States', 'state', 'target', 'speculate', 'incident', 'targeted', 'targeting', 'speculated', 'suicide', 'bomb', 'bombing', 'bomber', 'responsibility', 'claim', 'claimed', 'noted', 'State', 'carried', 'majority', 'minority'}
#consider making a single set

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in my_stopwords and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [132]:
stemmer = SnowballStemmer('english')
processed_docs = documents['motive'].map(preprocess)

In [133]:
processed_docs[:70]

3      [conrado, oligo, agtarap, run, town, councilor]                                                                                                                                                                                                                                 
6      [abdulaziz, musab, shabaab, spokesperson, victim, crime, muslim]                                                                                                                                                                                                                
7      [black, star, state, support, imprison, black, star, member]                                                                                                                                                                                                                    
8      [posit, maoist, accus, shyam, sikaka, polic, inform]                                                                                                     

In [134]:
### Bag of words

In [135]:
#create corpus
texts = [[''.join(item) for item in document] for document in processed_docs]
# texts = processed_docs

#create dictionary
dictionary = corpora.Dictionary(texts)
# id2word = gensim.corpora.Dictionary(processed_docs)

#Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in texts]

# bow_corpus = [id2word.doc2bow(text) for text in texts] 
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
 [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 2), (14, 1), (15, 1), (16, 2), (17, 1), (18, 1)],
 [(19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)],
 [(11, 1),
  (12, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1)],
 [(46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)],
 [(52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1)],
 [(49, 1),
  (54, 2),
  (55, 2),
  (56, 1),
  (57, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 1)],
 [(54, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1)],
 [(54, 1), (55, 1), (56, 1), (57, 1), (59, 1), (81, 1), (82, 1), (83, 1)],
 [(84, 1), (85,

In [136]:
model = ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
# lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=2, id2word=id2word, passes=2, workers=2)

#get docs and topics matrix: should be same # rows as there are motive data: 
# https://stackoverflow.com/questions/43357247/get-document-topics-and-get-term-topics-in-gensim

# tune the model by adjusting alpha or maybe beta, in order to make the words really distinct/far apart

In [137]:
# bow = lda_model.id2word.doc2bow(texts) # convert to bag of words format first
# gamma, phis, other = lda_model.get_document_topics(bow, 

topics = model.show_topics()
# lda_model.show_topics()
topics

[(0,
  '0.021*"shiit" + 0.020*"iraq" + 0.018*"sunni" + 0.017*"larger" + 0.017*"violenc" + 0.016*"retali" + 0.015*"communiti" + 0.014*"sectarian" + 0.013*"trend" + 0.011*"govern"'),
 (1,
  '0.012*"islam" + 0.011*"elect" + 0.010*"polic" + 0.010*"suspect" + 0.009*"iraq" + 0.008*"violenc" + 0.008*"relat" + 0.008*"kill" + 0.007*"larger" + 0.007*"state"')]

In [138]:
from pprint import pprint

# Print the Keyword in the 2 topics
#probability that the term belongs to the topic; prob of seeing that word given that topic
pprint(model.print_topics())
doc_lda = model[corpus]

[(0,
  '0.021*"shiit" + 0.020*"iraq" + 0.018*"sunni" + 0.017*"larger" + '
  '0.017*"violenc" + 0.016*"retali" + 0.015*"communiti" + 0.014*"sectarian" + '
  '0.013*"trend" + 0.011*"govern"'),
 (1,
  '0.012*"islam" + 0.011*"elect" + 0.010*"polic" + 0.010*"suspect" + '
  '0.009*"iraq" + 0.008*"violenc" + 0.008*"relat" + 0.008*"kill" + '
  '0.007*"larger" + 0.007*"state"')]


In [139]:
### Compute Model Perplexity and Coherence Score (interpretability of the model)

In [140]:
# Compute Perplexity
print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.88720274894677

Coherence Score:  0.22641622526063013


In [141]:
### Visualize the topics-keywords

In [142]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary)
vis

/Users/janestout/anaconda2/envs/py3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
0      0.03614  0.0  1       1        58.947449
1     -0.03614  0.0  2       1        41.052551, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
284   Default  95.000000   elect          95.000000   30.0000  30.0000
22    Default  97.000000   polic          97.000000   29.0000  29.0000
71    Default  151.000000  communiti      151.000000  28.0000  28.0000
406   Default  65.000000   relat          65.000000   27.0000  27.0000
78    Default  187.000000  sunni          187.000000  26.0000  26.0000
59    Default  168.000000  retali         168.000000  25.0000  25.0000
77    Default  224.000000  shiit          224.000000  24.0000  24.0000
75    Default  146.000000  sectarian      146.000000  23.0000  23.0000
266   Default  32.000000   schedul        32.000000   22.0000  22.0000
56    Default  149.000000  islam          149.000000  21.0000  21.0000
106   Default  79.000000   kill           79.000000   20.0000  20.0000
133   Default  28.000000   fight          28.000000   19.0000  19.0000
60    Default  50.000000   attempt        50.000000   18.0000  18.0000
404   Default  23.000000   januari        23.000000   17.0000  17.0000
21    Default  53.000000   maoist         53.000000   16.0000  16.0000
241   Default  28.000000   begin          28.000000   15.0000  15.0000
285   Default  26.000000   general        26.000000   14.0000  14.0000
405   Default  17.000000   nationwid      17.000000   13.0000  13.0000
409   Default  42.000000   intimid        42.000000   12.0000  12.0000
265   Default  19.000000   poll           19.000000   11.0000  11.0000
514   Default  23.000000   leav           23.000000   10.0000  10.0000
244   Default  16.000000   day            16.000000   9.0000   9.0000 
192   Default  16.000000   process        16.000000   8.0000   8.0000 
403   Default  20.000000   hartal         20.000000   7.0000   7.0000 
259   Default  30.000000   april          30.000000   6.0000   6.0000 
456   Default  17.000000   parliamentari  17.000000   5.0000   5.0000 
184   Default  22.000000   report         22.000000   4.0000   4.0000 
94    Default  53.000000   recent         53.000000   3.0000   3.0000 
194   Default  23.000000   tamil          23.000000   2.0000   2.0000 
282   Default  61.000000   occur          61.000000   1.0000   1.0000 
...       ...        ...     ...                ...      ...      ... 
106   Topic2   47.400455   kill           79.552017   0.3725  -4.8874 
409   Topic2   27.576582   intimid        42.731983   0.4523  -5.4291 
56    Topic2   78.205795   islam          149.029114  0.2455  -4.3867 
402   Topic2   15.374916   bangladesh     22.201881   0.5229  -6.0133 
169   Topic2   21.462536   hold           34.196434   0.4245  -5.6798 
282   Topic2   33.633965   occur          61.644142   0.2845  -5.2305 
431   Topic2   20.580629   anti           33.347782   0.4077  -5.7217 
98    Topic2   63.816860   suspect        157.936081 -0.0159  -4.5901 
109   Topic2   40.864090   area           86.587494   0.1394  -5.0358 
412   Topic2   17.141329   author         26.493038   0.4549  -5.9046 
479   Topic2   30.035925   peopl          58.250805   0.2280  -5.3437 
12    Topic2   41.376312   victim         95.224518   0.0568  -5.0234 
17    Topic2   45.688969   state          110.709732  0.0053  -4.9242 
289   Topic2   39.422119   pakistan       92.666016   0.0356  -5.0717 
95    Topic2   44.061852   taliban        136.678192 -0.2417  -4.9605 
187   Topic2   45.317665   govern         148.760269 -0.2983  -4.9324 
80    Topic2   53.231308   violenc        204.817978 -0.4572  -4.7714 
54    Topic2   56.534061   iraq           238.478714 -0.5491  -4.7112 
155   Topic2   39.090759   group          122.069481 -0.2484  -5.0802 
135   Topic2   31.604940   respons        79.838158  -0.0364  -5.2928 
26

In [17]:
# Source: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# So far you have seen Gensim’s inbuilt version of the LDA algorithm. Mallet’s version, however, often gives a better quality of topics. 
# Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself. 
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# install java development kit: https://www.oracle.com/technetwork/java/javase/downloads/jdk11-downloads-5066655.html

mallet_path = '/Users/janestout/Dropbox/Galvanize/DSI/Capstones/Capstone2_working/GTD/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)

In [18]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=processed_docs, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('area', 0.05716303470540759),
   ('intimid', 0.032728006456820016),
   ('govern', 0.031638418079096044),
   ('liber', 0.020258272800645682),
   ('aqap', 0.018523002421307506),
   ('peninsula', 0.01828087167070218),
   ('suspect', 0.017554479418886198),
   ('arabian', 0.016707021791767553),
   ('author', 0.016646489104116223),
   ('peac', 0.015778853914447135)]),
 (1,
  [('taliban', 0.10819794657959954),
   ('pakistan', 0.07299083434955352),
   ('kill', 0.06299909523622202),
   ('member', 0.04173714645371936),
   ('tehrik', 0.03939656189764368),
   ('respons', 0.036013532119114115),
   ('claim', 0.027654301561700956),
   ('spokesperson', 0.02730026356162228),
   ('death', 0.021556980449234885),
   ('strike', 0.02088824200464183)]),
 (2,
  [('claim', 0.10763166755798523),
   ('respons', 0.09758947847960861),
   ('group', 0.08649751421157922),
   ('unit', 0.04464515617881831),
   ('militari', 0.04032721294590687),
   ('state', 0.03765325727414978),
   ('civilian', 0.0242240576782

NameError: name 'id2word' is not defined

In [19]:


def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [20]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=bow_corpus, texts=processed_docs, start=2, limit=10, step=2)

NameError: name 'id2word' is not defined

In [ ]:
# Show graph
limit=10; start=2; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))